In [1]:
from dodfminer import ContentExtractor
from dodfminer import ActsExtractor
from dodfminer import downloader
from dodfminer.downloader.core import Downloader
import pandas as pd
import numpy as np
import glob
import re
import os
import nltk
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

# Dataframe

In [2]:
txt_files = glob.glob('../dodfs/**/**/**/*.txt')
txt_files.extend(glob.glob('../dodfs/**/**/*.txt'))

In [3]:
COLUNAS = [
    "Tipo do Ato",
    'Id Ato',
    "DODF", 
    "Data Ato/DODF",
    "Processo GDF/SEI",
    "Tipo",
    "Modo de Disputa",
    "Valor Estimado",
    "Redator",
    "Texto",
]

data = pd.DataFrame(columns=COLUNAS)

In [4]:
data

,Tipo do Ato,Id Ato,DODF,Data Ato/DODF,Processo GDF/SEI,Tipo,Modo de Disputa,Valor Estimado,Redator,Texto


In [5]:
for txt in txt_files:
    df = ActsExtractor.get_act_df("licitacao_abertura", txt, 'regex')
    for idx in range(len(df)):
        df['DODF'] = txt[:-4]
        df['Redator'][idx] = re.findall(r"<END_OF_BLOCK>[\s\S][A-Z \\n]+[\s\S]<END_OF_BLOCK>", df['Texto'][idx])[0]
        if pd.isna(df['Data Ato/DODF'][idx]):
            df['Data Ato/DODF'][idx] = re.findall(r'\d{2}-\d{2}-\d{2,4}', df['DODF'][idx])[0]
            if len(df['Data Ato/DODF'][idx]) == 8:
                df['Data Ato/DODF'][idx] = df['Data Ato/DODF'][idx][:6] + '20' + df['Data Ato/DODF'][idx][6:]
    data = pd.concat([data, df], ignore_index=True)

for idx in range(len(data)):
    data['Texto'][idx] = data['Tipo do Ato'][idx] + data['Texto'][idx]
    data['Texto'][idx] = data['Texto'][idx].replace('\n<END_OF_BLOCK>\n', ' ').replace('\n<END_OF_BLOCK>', ' ').replace('  ', ' ')
    data['Texto'][idx] = data['Texto'][idx].replace('\n', ' ')

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [6]:
len(data)

1243

In [7]:
def clean_process():
    for idx in range(len(data)):
        if not pd.isna(data['Processo GDF/SEI'][idx]):
            data['Processo GDF/SEI'][idx] = re.sub('[^0-9]', '', data['Processo GDF/SEI'][idx])

In [8]:
clean_process()

In [9]:
for idx in range(len(data)):
    if not pd.isna(data['Processo GDF/SEI'][idx]):
        data['Texto'][idx] = re.sub(r"""(?:(?:(?:P|p)rocesso(?:\s+)?(?:(?:()?SEI(?:)?))?(?:\s+)?(?:(?:no|n.o)?)?)|(?:P|p)rocesso:|(?:P|p)rocesso|Processo.|(?:P|p)rocesso\s+no|(?:P|p)rocesso\s+n.? ?o.?|(?:P|p)rocesso\s+no:|(?:P|p)rocesso\s+SEI\s+no:|(?:P|p)rocesso\s+SEI:|(?:P|p)rocesso\s+SEI-GDF:|(?:P|p)rocesso\s+SEI-GDF|(?:P|p)rocesso\s+SEI\s+no|(?:P|p)rocesso\s+SEI\s+n|(?:P|p)rocesso\s+SEI|(?:P|p)rocesso-\s+SEI|(?:P|p)rocesso\s+SEI\s+no.|(?:P|p)rocesso\s+(SEI)\s+no.|(?:P|p)rocesso\s+SEI.|(?:P|p)rocesso\s+(SEI-DF)\s+no.|(?:P|p)rocesso\s+SEI-GDF no|(?:P|p)rocesso\s+n|(?:P|p)rocesso\s+N|(?:P|p)rocesso\s+administrativo no|(?:P|p)rocesso\s+n:|PROCESSO: ?N?o?|PROCES-? ?SO|PROCESSO.|PROCESSO\s+no|PROCESSO\s+No|PROCESSO\s+N.o:?|PROCESSO\s+no.|PROCESSO\s+no:|PROCESSO\s+No:|PROCESSO\s+SEI\s+no:|PROCESSO\s+SEI:|PROCESSO\s+SEI|PROCESSO\s+SEI-GDF:|PROCESSO\s+SEI-GDF|PROCESSO\s+SEI\s+no|PROCESSO\s+SEI\s+No|PROCESSO\s+SEI\s+no.|PROCESSO\s+SEI.)((?:(?!\s\d{2}.\d{3}.\d{3}/\d{4}-\d{2}))(?:(?:\s)(?:(?:[\d.]+)|(?:[\d\s]+))[.-]?(?:(?:\d)|(?:[.\d\sSEI-|]+))(?:/|-
\b)(?:(?:(?:\d)+|(?:[\d\s]+)))?(?:-(?:(?:\d)+|(?:[\d\s]+)))?(?:-SECOM/DF|-?CEB|/CBMDF|F J Z B / D F)?))""", 'Processo: ' + data['Processo GDF/SEI'][idx], data['Texto'][idx])

In [10]:
data

,Tipo do Ato,Id Ato,DODF,Data Ato/DODF,Processo GDF/SEI,Tipo,Modo de Disputa,Valor Estimado,Redator,Texto
0,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2004\01_Janeiro\DODF 017 26-01-2004\D...,23 de janeiro de 2004,NaN,NaN,NaN,NaN,<END_OF_BLOCK>\nSECRETARIA DE ESTADO DE TRANSP...,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2004\01_Janeiro\DODF 021 30-01-2004\D...,29 de janeiro de 2004,NaN,NaN,NaN,NaN,<END_OF_BLOCK>\nCOMPANHIA ENERGETICA DE BRASIL...,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2004\02_Fevereiro\DODF 024 04-02-2004...,04-02-2004,001018682003,NaN,NaN,NaN,<END_OF_BLOCK>\nSECRETARIA DE ESTADO DE FAZEND...,AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2004\02_Fevereiro\DODF 037 25-02-2004...,25-02-2004,0010013122003,NaN,NaN,NaN,<END_OF_BLOCK>\nSECRETARIA DE ESTADO DE FAZEND...,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2004\02_Fevereiro\DODF 037 25-02-2004...,25-02-2004,0010013122003,NaN,NaN,NaN,<END_OF_BLOCK>\nSECRETARIA DE ESTADO DE FAZEND...,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
...,...,...,...,...,...,...,...,...,...,...
1238,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2021\08_Agosto\DODF 161 25-08-2021 1,24 de agosto de 2021,0005000035266202049,Menor Preco,Aberto,"R$ 3.864.574,44",<END_OF_BLOCK>\nAMILCAR UBIRATAN URACH VIEIRA\...,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1239,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2021\08_Agosto\DODF 161 25-08-2021 1,25-08-2021,0007200000930202199,Menor Preco por item,NaN,NaN,<END_OF_BLOCK>\nJOAO DE DEUS ABREU SOARES\n<EN...,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1240,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2021\08_Agosto\DODF 162 26-08-2021 1,26-08-2021,0402600022135202022,Menor Preco,NaN,"R$ R$ 19.940,00",<END_OF_BLOCK>\nROSIMEIRE PAIVA DA SILVA\n<END...,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...
1241,AVISO DE ABERTURA DE LICITACAO,NaN,../dodfs\2021\08_Agosto\DODF 162 26-08-2021 1,26-08-2021,0007200000455202151,Menor Preco por Grupo,NaN,NaN,<END_OF_BLOCK>\nJOAO DE DEUS ABREU SOARES\n<EN...,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...


# Preprocess

In [11]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


class CRFFeaturizer(BaseEstimator, TransformerMixin):

    @classmethod
    def crf_sentence_featurizer(cls, tokens: Iter[str]):
        """Generates features from sequence of tokens.

        Args:
            tokens (Iter[str]): [description]

        Returns:
            List[Dict]: list of dicts containing the feature of each token
        """
        return [{
                'word': token.lower(),
                'capital_letter': token[0].isupper(),
                'all_capital': token.isupper(),
                'isdigit': token.isdigit(),
                'word_before': token.lower() if j==0 else tokens[j-1].lower(),
                'word_after:': token.lower() if j+1>=len(tokens) else tokens[j+1].lower(),
                'BOS': j==0,
                'EOS': j==len(tokens)-1
            }
            for j, token in enumerate(tokens) if token
        ]



    def __init__(self, sent_featurizer: Callable = None,):
        self.text_matrix  = []
        self.sent_featurizer = sent_featurizer or self.crf_sentence_featurizer


    def fit(self, X = None, y=None, **fit_params):
        return self


    def transform(self, X: Iter[Iter[str]] ):
        if not isinstance(X, pd.Series):
            X = pd.Series(X)
        X = X.map(self.sent_featurizer)
        return X.values

In [12]:
class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)

# Dataframe com as entidade selecionadas

In [13]:
df = data[['Tipo do Ato', 'Data Ato/DODF', 'Processo GDF/SEI', 'Tipo', 'Modo de Disputa', 'Valor Estimado', 'Redator', 'Texto']]

df.rename({'Tipo do Ato': 'ATO', 'Data Ato/DODF': 'DATA', 'Processo GDF/SEI': 'PROCESSO', 'Tipo': 'TIPO', 'Modo de Disputa': 'DISPUTA', 'Valor Estimado': 'V_ESTIMADO', 'Redator': 'REDATOR'}, axis=1, inplace=True)

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
for idx in range(len(df)):
    if type(df['TIPO'][idx]) != float:
        df['TIPO'][idx] = df['TIPO'][idx].replace('\n<END_OF_BLOCK>\n', ' ').replace('\n<END_OF_BLOCK>', ' ').replace('  ', ' ')
        df['TIPO'][idx] = df['TIPO'][idx].replace('\n', ' ')
        if df['TIPO'][idx] == ' ' or df['TIPO'][idx] == '':
            df['TIPO'][idx] = np.nan
    if type(df['DISPUTA'][idx]) != float:
        df['DISPUTA'][idx] = df['DISPUTA'][idx].replace('\n<END_OF_BLOCK>\n', ' ').replace('\n<END_OF_BLOCK>', ' ').replace('  ', ' ')
        df['DISPUTA'][idx] = df['DISPUTA'][idx].replace('\n', ' ')
        if df['DISPUTA'][idx] == ' ' or df['DISPUTA'][idx] == '':
            df['DISPUTA'][idx] = np.nan
    if type(df['V_ESTIMADO'][idx]) != float:
        df['V_ESTIMADO'][idx] = df['V_ESTIMADO'][idx].replace('\n<END_OF_BLOCK>\n', ' ').replace('\n<END_OF_BLOCK>', ' ').replace('  ', ' ')
        df['V_ESTIMADO'][idx] = df['V_ESTIMADO'][idx].replace('\n', ' ').replace('R$ R$', 'R$')
        if df['V_ESTIMADO'][idx] == ' ' or df['V_ESTIMADO'][idx] == '' or df['V_ESTIMADO'][idx] == '  ':
            df['V_ESTIMADO'][idx] = np.nan
    if type(df['REDATOR'][idx]) != float:
        df['REDATOR'][idx] = df['REDATOR'][idx].replace('<END_OF_BLOCK>\n', '').replace('\n<END_OF_BLOCK>', '')
        if df['REDATOR'][idx] == ' ' or df['REDATOR'][idx] == '' or 'COMPANHIA' in df['REDATOR'][idx] or 'SECRETARIA' in df['REDATOR'][idx]:
            df['REDATOR'][idx] = np.nan

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
df

,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,REDATOR,Texto
0,AVISO DE ABERTURA DE LICITACAO,23 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1,AVISO DE ABERTURA DE LICITACAO,29 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2,AVISO DE ABERTURA DE LICITACAO,04-02-2004,001018682003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3,AVISO DE ABERTURA DE LICITACAO,25-02-2004,0010013122003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4,AVISO DE ABERTURA DE LICITACAO,25-02-2004,0010013122003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
...,...,...,...,...,...,...,...,...
1238,AVISO DE ABERTURA DE LICITACAO,24 de agosto de 2021,0005000035266202049,Menor Preco,Aberto,"R$ 3.864.574,44",AMILCAR UBIRATAN URACH VIEIRA,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1239,AVISO DE ABERTURA DE LICITACAO,25-08-2021,0007200000930202199,Menor Preco por item,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1240,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0402600022135202022,Menor Preco,NaN,"R$ 19.940,00",ROSIMEIRE PAIVA DA SILVA,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...
1241,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0007200000455202151,Menor Preco por Grupo,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...


In [16]:
# df[['ATO', 'DATA', 'PROCESSO', 'TIPO', 'DISPUTA', 'V_ESTIMADO']].loc[df['DATA'] == '08 de agosto de 2016']

In [21]:
df.iloc[[949]]

,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,REDATOR,Texto
949,AVISO DE ABERTURA DE LICITACAO,18 de junho de 2020,0011100000742201941,Menor Preco Objeto: Elaboracao dos Projetos E...,NaN,NaN,COMISSAO DE VENDA DIRETA,AVISO DE ABERTURA DE LICITACAO O Presidente da...


In [22]:
df.isnull().sum()

ATO              0
DATA             0
PROCESSO       277
TIPO           582
DISPUTA       1175
V_ESTIMADO     770
REDATOR        331
Texto            0
dtype: int64

# IOBifyer

In [23]:
iob = IOBifyer(column='Texto')
r = iob.transform(df)

In [24]:
r

0       [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1       [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
2       [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
3       [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
4       [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
                              ...                        
1238    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1239    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1240    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1241    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
1242    [B-ATO, I-ATO, I-ATO, I-ATO, I-ATO, O, O, O, O...
Length: 1243, dtype: object

In [25]:
df.shape, r.shape

((1243, 8), (1243,))

In [26]:
df["IOB"] = np.nan

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
for idx in range(len(df)):
    df["IOB"][idx] = ' '.join(r[idx])

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [28]:
df

,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,REDATOR,Texto,IOB
0,AVISO DE ABERTURA DE LICITACAO,23 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
1,AVISO DE ABERTURA DE LICITACAO,29 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
2,AVISO DE ABERTURA DE LICITACAO,04-02-2004,001018682003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
3,AVISO DE ABERTURA DE LICITACAO,25-02-2004,0010013122003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
4,AVISO DE ABERTURA DE LICITACAO,25-02-2004,0010013122003,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
...,...,...,...,...,...,...,...,...,...
1238,AVISO DE ABERTURA DE LICITACAO,24 de agosto de 2021,0005000035266202049,Menor Preco,Aberto,"R$ 3.864.574,44",AMILCAR UBIRATAN URACH VIEIRA,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1239,AVISO DE ABERTURA DE LICITACAO,25-08-2021,0007200000930202199,Menor Preco por item,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1240,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0402600022135202022,Menor Preco,NaN,"R$ 19.940,00",ROSIMEIRE PAIVA DA SILVA,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O B-...
1241,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0007200000455202151,Menor Preco por Grupo,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...


In [29]:
# df['Texto'][0]

In [30]:
len(DEFAULT_TOKENIZER(df['Texto'][0]))

146

In [31]:
len(df['IOB'][0].split())

146

In [32]:
# df['IOB'][0]

In [34]:
df.tail()

,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,REDATOR,Texto,IOB
1238,AVISO DE ABERTURA DE LICITACAO,24 de agosto de 2021,0005000035266202049,Menor Preco,Aberto,"R$ 3.864.574,44",AMILCAR UBIRATAN URACH VIEIRA,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1239,AVISO DE ABERTURA DE LICITACAO,25-08-2021,0007200000930202199,Menor Preco por item,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1240,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0402600022135202022,Menor Preco,NaN,"R$ 19.940,00",ROSIMEIRE PAIVA DA SILVA,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O B-...
1241,AVISO DE ABERTURA DE LICITACAO,26-08-2021,0007200000455202151,Menor Preco por Grupo,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1242,AVISO DE ABERTURA DE LICITACAO,30 de agosto de 2021,0402600012872202106,Menor Preco,NaN,NaN,VALTER CASIMIRO SILVEIRA,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...


# Formato deep learning

In [84]:
# df['iob'] = np.nan

In [85]:
# for row in range(len(df)):
#     lista_IOB = []
#     for idx in range(len(DEFAULT_TOKENIZER(df['Texto'][row]))):
#         texto = f"{DEFAULT_TOKENIZER(df['Texto'][row])[idx]} -X- -X- {df['IOB'][row].split()[idx]}"
#         lista_IOB.append(texto)
#     df['iob'][row] = '\n'.join(lista_IOB)


In [86]:
# df['Texto'][0]

In [87]:
# df['iob'][0]

In [88]:
# df['IOB'][0].split()[0]

In [89]:
# df[['ATO', 'DATA', 'PROCESSO', 'Texto', 'IOB', 'iob']].loc[df['DATA'] == '29 de marco de 2010']

In [90]:
# df['Texto'][174]

In [91]:
# df

In [92]:
# df2 = df[['iob']]
# df2

In [93]:
# df2.to_csv('CSVs/dl_IOB.txt', header=False, sep='\n', index=False)

In [94]:
# df.isnull().sum()

# Salvando CSV

In [36]:
df.to_csv('CSVs/IOBabertura.csv')